<a href="https://colab.research.google.com/github/ThaniaKira/CSST101-3B-AT2526/blob/main/Bulos_Assessment_No_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Part 1 - Unification**

In [ ]:
def unify(p1, p2):
    f1, f2 = p1.split("(")[0], p2.split("(")[0]
    if f1 != f2: return "Cannot be unified"

    a1 = p1[p1.find("(")+1:-1].split(",")
    a2 = p2[p2.find("(")+1:-1].split(",")
    if len(a1) != len(a2): return "Cannot be unified"

    subs = {}
    for x, y in zip(a1, a2):
        x, y = x.strip(), y.strip()
        if x == y: continue
        if x.islower(): subs[x] = y
        elif y.islower(): subs[y] = x
        else: return "Cannot be unified"

    return subs or "Already unified"

result = unify("Eligible(x)", "Eligible(Ana)")
print(result)

{'x': 'Ana'}


# **Part 2 - Forward Chaining**

In [10]:
facts = {
    "Student(Juan)",
    "HighGPA(Juan)",
    "Student(Ana)",
    "HighGPA(Ana)",
    "Student(Maria)",
    "Student(Carlos)"
}

rules = [
    ("Student", "Qualified"),
    ("HighGPA", "Eligible"),
    ("Eligible", "Scholarship")
]

print("Initial facts:", facts)

for premise, conclusion in rules:
    print(f"\nApplying Rule: IF {premise}(x) THEN {conclusion}(x)")
    new_facts = set()
    for fact in list(facts): # Iterate over a copy
        if fact.startswith(premise + "("):
            name = fact[fact.find("(")+1:-1]
            new_fact = f"{conclusion}({name})"
            if new_fact not in facts:
                new_facts.add(new_fact)
                print(f"  Derived: {new_fact} from {fact}")
            else:
                print(f"  {new_fact} already exists in facts.")

    facts.update(new_facts)
    print(f"  Current facts: {facts}")


print(f"\nFinal facts: {facts}")

print("\n" + "="*50)
print("ANALYSIS:")
print("Juan:", "Eligible" if "Eligible(Juan)" in facts else "Not eligible")
print("Ana:", "Eligible" if "Eligible(Ana)" in facts else "Not eligible")
print("Maria:", "Eligible" if "Eligible(Maria)" in facts else "Not eligible")
print("Carlos:", "Eligible" if "Eligible(Carlos)" in facts else "Not eligible")

print("\nREASON:")
print("Maria and Carlos are not eligible and do not have Scholarship because they don't have High GPA.")
print("Juan and Ana are eligible and have Scholarship because they have High GPA.")

Initial facts: {'HighGPA(Juan)', 'Student(Ana)', 'Student(Maria)', 'HighGPA(Ana)', 'Student(Juan)', 'Student(Carlos)'}

Applying Rule: IF Student(x) THEN Qualified(x)
  Derived: Qualified(Ana) from Student(Ana)
  Derived: Qualified(Maria) from Student(Maria)
  Derived: Qualified(Juan) from Student(Juan)
  Derived: Qualified(Carlos) from Student(Carlos)
  Current facts: {'Student(Ana)', 'Qualified(Juan)', 'Qualified(Maria)', 'Student(Maria)', 'HighGPA(Juan)', 'HighGPA(Ana)', 'Student(Juan)', 'Student(Carlos)', 'Qualified(Ana)', 'Qualified(Carlos)'}

Applying Rule: IF HighGPA(x) THEN Eligible(x)
  Derived: Eligible(Juan) from HighGPA(Juan)
  Derived: Eligible(Ana) from HighGPA(Ana)
  Current facts: {'Student(Ana)', 'Qualified(Juan)', 'Qualified(Maria)', 'Student(Maria)', 'HighGPA(Juan)', 'HighGPA(Ana)', 'Student(Juan)', 'Student(Carlos)', 'Qualified(Ana)', 'Qualified(Carlos)', 'Eligible(Juan)', 'Eligible(Ana)'}

Applying Rule: IF Eligible(x) THEN Scholarship(x)
  Derived: Scholarship(Jua

# **Part 3 - Backward Chaining**

In [12]:
# Facts and rules
facts = {"Student(Carlos)", "Studies(Carlos)", "Submits(Carlos, IT101)"}
premises = ["Studies(x)", "Submits(x, IT101)"]
conclusion = "Passed(x, IT101)"

def prove(goal):
    print(f"Trying to prove: {goal}")

    if goal in facts:
        print(f"✓ Found in facts: {goal}")
        return True

    if goal.startswith("Passed("):
        name = goal[goal.find("(")+1:goal.find(",")]
        course = goal[goal.find(",")+1:goal.find(")")].strip()
        print(f"Need to prove Passed({name}, {course}) - checking premises...")

        premises_proven = []
        for premise in premises:
            subgoal = premise.replace("x", name)
            if "," in premise:  # Handle predicates with multiple arguments
                subgoal = subgoal.replace("IT101", course)
            print(f"  Proving subgoal: {subgoal}")
            result = prove(subgoal)
            premises_proven.append(result)

        final_result = all(premises_proven)
        print(f"All premises for {goal} proven? {final_result}")
        return final_result

    print(f"✗ Cannot prove: {goal} (not in facts and no rule matches)")
    return False

print("=" * 50)
print("GOAL 1: Scholarship(Maria)")
print("=" * 50)
# For Scholarship(Maria), we need appropriate facts and rules
# Since we don't have rules/facts about Scholarship, it will fail
scholarship_facts = {"Student(Maria)"} # Modified facts to exclude Hardworking(Maria)
scholarship_rules = [("Student(x) & Hardworking(x)", "Scholarship(x)")]

def prove_scholarship(goal):
    print(f"Trying to prove: {goal}")

    if goal in scholarship_facts:
        print(f"✓ Found in facts: {goal}")
        return True

    if goal.startswith("Scholarship("):
        name = goal[goal.find("(")+1:-1]
        print(f"Need to prove Scholarship({name}) - checking rule premises...")

        # Check if Student and Hardworking
        student_goal = f"Student({name})"
        hardworking_goal = f"Hardworking({name})"

        print(f"  Proving subgoal: {student_goal}")
        student_proven = prove_scholarship(student_goal)

        print(f"  Proving subgoal: {hardworking_goal}")
        hardworking_proven = prove_scholarship(hardworking_goal)

        final_result = student_proven and hardworking_proven
        print(f"All premises for {goal} proven? {final_result}")
        return final_result

    print(f"✗ Cannot prove: {goal}")
    return False

print("Scholarship(Maria)?", prove_scholarship("Scholarship(Maria)"))
print("\n" + "=" * 50)

print("GOAL 2: FinancialSupport(Ana)")
print("=" * 50)
# For FinancialSupport(Ana), we need appropriate facts and rules
financial_facts = {"Student(Ana)", "LowIncome(Ana)", "Applies(Ana, FinancialAid)"}
financial_rules = [("Student(x) & LowIncome(x) & Applies(x, FinancialAid)", "FinancialSupport(x)")]

def prove_financial(goal):
    print(f"Trying to prove: {goal}")

    if goal in financial_facts:
        print(f"✓ Found in facts: {goal}")
        return True

    if goal.startswith("FinancialSupport("):
        name = goal[goal.find("(")+1:-1]
        print(f"Need to prove FinancialSupport({name}) - checking rule premises...")

        # Check all conditions
        student_goal = f"Student({name})"
        income_goal = f"LowIncome({name})"
        applies_goal = f"Applies({name}, FinancialAid)"

        print(f"  Proving subgoal: {student_goal}")
        student_proven = prove_financial(student_goal)

        print(f"  Proving subgoal: {income_goal}")
        income_proven = prove_financial(income_goal)

        print(f"  Proving subgoal: {applies_goal}")
        applies_proven = prove_financial(applies_goal)

        final_result = student_proven and income_proven and applies_proven
        print(f"All premises for {goal} proven? {final_result}")
        return final_result

    print(f"✗ Cannot prove: {goal}")
    return False

print("FinancialSupport(Ana)?", prove_financial("FinancialSupport(Ana)"))

GOAL 1: Scholarship(Maria)
Trying to prove: Scholarship(Maria)
Need to prove Scholarship(Maria) - checking rule premises...
  Proving subgoal: Student(Maria)
Trying to prove: Student(Maria)
✓ Found in facts: Student(Maria)
  Proving subgoal: Hardworking(Maria)
Trying to prove: Hardworking(Maria)
✗ Cannot prove: Hardworking(Maria)
All premises for Scholarship(Maria) proven? False
Scholarship(Maria)? False

GOAL 2: FinancialSupport(Ana)
Trying to prove: FinancialSupport(Ana)
Need to prove FinancialSupport(Ana) - checking rule premises...
  Proving subgoal: Student(Ana)
Trying to prove: Student(Ana)
✓ Found in facts: Student(Ana)
  Proving subgoal: LowIncome(Ana)
Trying to prove: LowIncome(Ana)
✓ Found in facts: LowIncome(Ana)
  Proving subgoal: Applies(Ana, FinancialAid)
Trying to prove: Applies(Ana, FinancialAid)
✓ Found in facts: Applies(Ana, FinancialAid)
All premises for FinancialSupport(Ana) proven? True
FinancialSupport(Ana)? True


**REASONING STEPS:**

**Goal 1: Scholarship(Maria)**

1.   Check if "Scholarship(Maria)" is in facts → Not found
2.   Apply rule: IF Student(x) & Hardworking(x) THEN Scholarship(x)
1.   Check subgoal: "Student(Maria)" → ✓ Found in facts
2.   Check subgoal: "Hardworking(Maria)" → ✗ Not found in facts

**Conclusion**: Cannot prove Scholarship(Maria)


**Goal 2: FinancialSupport(Ana)**

1.   Check if "FinancialSupport(Ana)" is in facts → Not found
1.   Apply rule: IF Student(x) & LowIncome(x) & Applies(x, FinancialAid) THEN FinancialSupport(x)
2.   Check subgoal: "Student(Ana)" → ✓ Found in facts
2.   Check subgoal: "LowIncome(Ana)" → ✓ Found in facts
1.   Check subgoal: "Applies(Ana, FinancialAid)" → ✓ Found in facts

**Conclusion**: ✓ FinancialSupport(Ana) proven

# **Part 4 - Validity Check**

In [13]:
# Truth table for (HighGrades ∧ Leader ∧ Applied) → Eligible
print("Truth Table for (HighGrades ∧ Leader ∧ Applied) → Eligible")
print("HighGrades | Leader | Applied | Eligible")
print("-----------------------------------------")

# Generate all combinations
for hg in [False, True]:
    for ld in [False, True]:
        for ap in [False, True]:
            # Rule: (HighGrades AND Leader AND Applied) → Eligible
            eligible = (hg and ld and ap)
            print(f"{str(hg).ljust(9)} | {str(ld).ljust(6)} | {str(ap).ljust(7)} | {str(eligible).ljust(8)}")

print("\n" + "="*50)
print("Testing Students:")
print("="*50)

# Student facts
students = {
    "Juan": {"HighGrades": True, "Leader": True, "Applied": True},
    "Maria": {"HighGrades": True, "Leader": False, "Applied": True},
    "Carlos": {"HighGrades": False, "Leader": True, "Applied": True},
    "Ana": {"HighGrades": True, "Leader": True, "Applied": False}
}

# Test each student
for student, attributes in students.items():
    hg = attributes["HighGrades"]
    ld = attributes["Leader"]
    ap = attributes["Applied"]
    eligible = (hg and ld and ap)

    print(f"{student}: HighGrades={hg}, Leader={ld}, Applied={ap} → Eligible={eligible}")

Truth Table for (HighGrades ∧ Leader ∧ Applied) → Eligible
HighGrades | Leader | Applied | Eligible
-----------------------------------------
False     | False  | False   | False   
False     | False  | True    | False   
False     | True   | False   | False   
False     | True   | True    | False   
True      | False  | False   | False   
True      | False  | True    | False   
True      | True   | False   | False   
True      | True   | True    | True    

Testing Students:
Juan: HighGrades=True, Leader=True, Applied=True → Eligible=True
Maria: HighGrades=True, Leader=False, Applied=True → Eligible=False
Carlos: HighGrades=False, Leader=True, Applied=True → Eligible=False
Ana: HighGrades=True, Leader=True, Applied=False → Eligible=False


# **Part 5 - Reflection**

In AI, consistent data and accurate logic are fundamental, making logical reasoning indispensable. It enables transparent and explainable decisions based on established rules and facts. In real-life decision-making, logical reasoning helps identify optimal choices by minimizing biases and emotional influences, ensuring objectivity and reliability.